<a href="https://www.kaggle.com/code/joanwachuka/ksl-sign?scriptVersionId=293113824" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ksl-hand-landmarks/dataset/mother/mother_12.npy
/kaggle/input/ksl-hand-landmarks/dataset/mother/mother_9.npy
/kaggle/input/ksl-hand-landmarks/dataset/mother/mother_21.npy
/kaggle/input/ksl-hand-landmarks/dataset/mother/mother_5.npy
/kaggle/input/ksl-hand-landmarks/dataset/mother/mother_22.npy
/kaggle/input/ksl-hand-landmarks/dataset/mother/mother_16.npy
/kaggle/input/ksl-hand-landmarks/dataset/mother/mother_1.npy
/kaggle/input/ksl-hand-landmarks/dataset/mother/mother_11.npy
/kaggle/input/ksl-hand-landmarks/dataset/mother/mother_3.npy
/kaggle/input/ksl-hand-landmarks/dataset/mother/mother_10.npy
/kaggle/input/ksl-hand-landmarks/dataset/mother/mother_2.npy
/kaggle/input/ksl-hand-landmarks/dataset/mother/mother_18.npy
/kaggle/input/ksl-hand-landmarks/dataset/mother/mother_7.npy
/kaggle/input/ksl-hand-landmarks/dataset/mother/mother_14.npy
/kaggle/input/ksl-hand-landmarks/dataset/mother/mother_15.npy
/kaggle/input/ksl-hand-landmarks/dataset/mother/mother_8.npy
/kaggle/input/k

In [2]:
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout ,Normalization , BatchNormalization, Bidirectional, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

2026-01-21 11:22:41.942775: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768994562.144774      24 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768994562.199543      24 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768994562.690211      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768994562.690262      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768994562.690264      24 computation_placer.cc:177] computation placer alr

In [3]:

DATA_PATH = "/kaggle/input/ksl-hand-landmarks/dataset"
MAX_FRAMES = 60
FEATURES = 126
EPOCHS = 200  
BATCH_SIZE = 4
LEARNING_RATE = 0.001
LABELS = ['a', 'is', 'mother', 'my', 'this'] 

In [4]:
# Load data 
X, y = [], []
for label_idx, label in enumerate(LABELS):
    label_dir = os.path.join(DATA_PATH, label)
    for file in os.listdir(label_dir):
        if file.endswith('.npy'):
            seq = np.load(os.path.join(label_dir, file))
            X.append(seq)
            y.append(label_idx)


print("Number of sequences:", len(X))
print("Example sequence shape:", X[0].shape)

# Convert to arrays
X = np.array(X, dtype=object)
y = np.array(y)


# Pad sequences
X_padded = pad_sequences(X, maxlen=MAX_FRAMES, dtype='float32', padding='post', truncating='post')
y_onehot = to_categorical(y, num_classes=len(LABELS))
print("X shape after padding:", X_padded.shape)



Number of sequences: 101
Example sequence shape: (23, 126)
X shape after padding: (101, 60, 126)


In [5]:
# Train/val/test split (stratified)
X_train, X_temp, y_train, y_temp = train_test_split(
    X_padded, y_onehot, test_size=0.3, random_state=42, stratify=y
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=np.argmax(y_temp, axis=1)
)
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("x_val shape:", X_val.shape)



norm = Normalization(axis=-1)
norm.adapt(X_train)

X_train shape: (70, 60, 126)
X_test shape: (16, 60, 126)
x_val shape: (15, 60, 126)


I0000 00:00:1768994576.842112      24 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [6]:


model = Sequential([
  
        Input(shape=(MAX_FRAMES, FEATURES)),
    
        Bidirectional(LSTM(32, return_sequences=True)),
        
        LSTM(16, return_sequences=False, dropout=0.3),
        BatchNormalization(),
        
        Dense(1024, activation='relu'),
        Dropout(0.4),
        
        Dense(512, activation='relu'),
        Dropout(0.4),
        Dense(len(LABELS), activation='softmax')
    
    
])

model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

callbacks = [
    EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=8, min_lr=1e-6),
    ModelCheckpoint('best_model.keras', monitor='val_accuracy', save_best_only=True, mode='max')
]

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)   │ (None, 60, 64)         │        40,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 16)             │         5,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 16)             │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │        17,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 5)              │         2,565 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 590,725 (2.25 MB)

 Trainable params: 590,693 (2.25 MB)

 Non-trainable params: 32 (128.00 B)

In [7]:

history = model.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=callbacks,
    verbose=1
)


test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\n Test Accuracy: {test_acc:.4f}")


model.save("KSL_model.keras")
np.save("sign_labels.npy", np.array(LABELS))

print("Model and labels saved!")


Epoch 1/200


I0000 00:00:1768994582.440758      66 cuda_dnn.cc:529] Loaded cuDNN version 91002


18/18 ━━━━━━━━━━━━━━━━━━━━ 7s 49ms/step - accuracy: 0.1741 - loss: 1.6238 - val_accuracy: 0.2000 - val_loss: 1.6099 - learning_rate: 0.0010
Epoch 2/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.1126 - loss: 1.6278 - val_accuracy: 0.2000 - val_loss: 1.6094 - learning_rate: 0.0010
Epoch 3/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.2005 - loss: 1.6007 - val_accuracy: 0.2000 - val_loss: 1.6090 - learning_rate: 0.0010
Epoch 4/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.0615 - loss: 1.6268 - val_accuracy: 0.2000 - val_loss: 1.6088 - learning_rate: 0.0010
Epoch 5/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.2219 - loss: 1.6153 - val_accuracy: 0.2000 - val_loss: 1.6092 - learning_rate: 0.0010
Epoch 6/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.1338 - loss: 1.6154 - val_accuracy: 0.2000 - val_loss: 1.6098 - learning_rate: 0.0010
Epoch 7/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.1726 - loss: 1.6313 - val_accuracy

In [8]:
# Evaluate
print("\nEvaluating best model...")
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")

# Predict and show classification report
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

from sklearn.metrics import classification_report, confusion_matrix
print("\nClassification Report:")
print(classification_report(y_true_classes, y_pred_classes, target_names=LABELS))

print("\nConfusion Matrix:")
print(confusion_matrix(y_true_classes, y_pred_classes))


Evaluating best model...
Test Accuracy: 0.8125
Test Loss: 0.2665
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 283ms/step

Classification Report:
              precision    recall  f1-score   support

           a       1.00      1.00      1.00         4
          is       1.00      1.00      1.00         3
      mother       1.00      1.00      1.00         3
          my       0.00      0.00      0.00         3
        this       0.50      1.00      0.67         3

    accuracy                           0.81        16
   macro avg       0.70      0.80      0.73        16
weighted avg       0.72      0.81      0.75        16


Confusion Matrix:
[[4 0 0 0 0]
 [0 3 0 0 0]
 [0 0 3 0 0]
 [0 0 0 0 3]
 [0 0 0 0 3]]


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [9]:
import numpy as np
from collections import defaultdict

# Convert one-hot y_test back to integers
y_test_int = np.argmax(y_test, axis=1)

# Group indices by class in the test set
indices_by_class = defaultdict(list)
for idx, label in enumerate(y_test_int):
    indices_by_class[label].append(idx)

# Select up to 3 samples per class
selected_indices = []
for label in range(len(LABELS)):
    if label in indices_by_class:
        selected = indices_by_class[label][:3]  
        selected_indices.extend(selected)

# Extract those samples
X_demo = X_test[selected_indices]
y_demo_true = y_test_int[selected_indices]

# Predict
y_demo_pred = model.predict(X_demo)
y_demo_pred_labels = np.argmax(y_demo_pred, axis=1)

# Print results
print("Manual test on 3 samples per class :\n")
for i, idx in enumerate(selected_indices):
    true_label = LABELS[y_demo_true[i]]
    pred_label = LABELS[y_demo_pred_labels[i]]
    confidence = np.max(y_demo_pred[i])
    status = "CORRECT" if true_label == pred_label else "WRONG"
    print(f"{status} Sample {i+1}: True='{true_label}' | Pred='{pred_label}' | Conf={confidence:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 286ms/step
Manual test on 3 samples per class :

CORRECT Sample 1: True='a' | Pred='a' | Conf=1.00
CORRECT Sample 2: True='a' | Pred='a' | Conf=0.99
CORRECT Sample 3: True='a' | Pred='a' | Conf=1.00
CORRECT Sample 4: True='is' | Pred='is' | Conf=0.99
CORRECT Sample 5: True='is' | Pred='is' | Conf=0.99
CORRECT Sample 6: True='is' | Pred='is' | Conf=0.97
CORRECT Sample 7: True='mother' | Pred='mother' | Conf=1.00
CORRECT Sample 8: True='mother' | Pred='mother' | Conf=0.99
CORRECT Sample 9: True='mother' | Pred='mother' | Conf=1.00
WRONG Sample 10: True='my' | Pred='this' | Conf=0.52
WRONG Sample 11: True='my' | Pred='this' | Conf=0.53
WRONG Sample 12: True='my' | Pred='this' | Conf=0.58
CORRECT Sample 13: True='this' | Pred='this' | Conf=0.54
CORRECT Sample 14: True='this' | Pred='this' | Conf=0.51
CORRECT Sample 15: True='this' | Pred='this' | Conf=0.59
